In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/Belphegorus86/lab-imbalanced-data/master/files_for_lab/customer_churn.csv"

df = pd.read_csv(url)

print(df.head())

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

y = df['Churn'].map({'Yes': 1, 'No': 0}) 

X = df[['tenure', 'SeniorCitizen', 'MonthlyCharges']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
es
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

conf_matrix, class_report

(array([[957,  79],
        [192, 181]], dtype=int64),
 '              precision    recall  f1-score   support\n\n           0       0.83      0.92      0.88      1036\n           1       0.70      0.49      0.57       373\n\n    accuracy                           0.81      1409\n   macro avg       0.76      0.70      0.72      1409\nweighted avg       0.80      0.81      0.80      1409\n')

In [6]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

balance_check = y_train_smote.value_counts()
a
model_smote = LogisticRegression()
model_smote.fit(X_train_smote, y_train_smote)

y_pred_smote = model_smote.predict(X_test_scaled)

conf_matrix_smote = confusion_matrix(y_test, y_pred_smote)
class_report_smote = classification_report(y_test, y_pred_smote)

balance_check, conf_matrix_smote, class_report_smote


(Churn
 0    4138
 1    4138
 Name: count, dtype: int64,
 array([[762, 274],
        [ 85, 288]], dtype=int64),
 '              precision    recall  f1-score   support\n\n           0       0.90      0.74      0.81      1036\n           1       0.51      0.77      0.62       373\n\n    accuracy                           0.75      1409\n   macro avg       0.71      0.75      0.71      1409\nweighted avg       0.80      0.75      0.76      1409\n')

In [7]:
from imblearn.under_sampling import TomekLinks

tl = TomekLinks()
X_train_tl, y_train_tl = tl.fit_resample(X_train_scaled, y_train)

balance_check_tl = y_train_tl.value_counts()

model_tl = LogisticRegression()
model_tl.fit(X_train_tl, y_train_tl)

y_pred_tl = model_tl.predict(X_test_scaled)

conf_matrix_tl = confusion_matrix(y_test, y_pred_tl)
class_report_tl = classification_report(y_test, y_pred_tl)

balance_check_tl, conf_matrix_tl, class_report_tl


(Churn
 0    3725
 1    1496
 Name: count, dtype: int64,
 array([[914, 122],
        [168, 205]], dtype=int64),
 '              precision    recall  f1-score   support\n\n           0       0.84      0.88      0.86      1036\n           1       0.63      0.55      0.59       373\n\n    accuracy                           0.79      1409\n   macro avg       0.74      0.72      0.72      1409\nweighted avg       0.79      0.79      0.79      1409\n')